In [1]:
import csv
import os
import copy
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
import pickle as pkl
import matplotlib.pyplot as plt
import random
import statsmodels.api as sm
import warnings
import pymc as pm
from pytensor import tensor as pt 

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.filterwarnings('ignore')

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
gdp_regression_data = pd.read_csv("../data/regression/gdp_regression_data.csv").dropna().reset_index(drop=True)

In [ ]:
# TODO: add code to make the first fixed effect 0

In [3]:
model_spec = {
    "continuous_covariates" : [
        'fd_humidity_[weight]', 'fd_humidity_[weight]_2', 'fd_humidity_[weight]_3',
        'fd_precip_[weight]_3', 
        'fd_humidity_annual_std_[weight]', 'fd_humidity_annual_std_[weight]_2', 'fd_precip_[weight]', 
        'fd_precip_[weight]_2', 'fd_precip_annual_std_[weight]', 'fd_precip_daily_std_[weight]', 
        'fd_precip_daily_std_[weight]_2', 'fd_precip_daily_std_[weight]_3', 'fd_temp_annual_std_[weight]', 
        'fd_temp_annual_std_[weight]_2', 'fd_temp_annual_std_[weight]_3', 'fd_temp_daily_std_[weight]', 
        'fd_temp_daily_std_[weight]_2', 'humidity_daily_std_[weight]', 'precip_[weight]', 
        'precip_daily_std_[weight]', 'temp_[weight]', 'temp_[weight]_2', 
        'temp_[weight]_3'
    ],
    "discrete_covariates" : ['wildfire','drought','wildfire_drought'],
    "fixed_effects" : ["country","year"],
    "incremental_effects" : 3,
    "weights" : "unweighted",
    "target" : "fd_ln_gdp"
}

# Add scaled continuous data

In [4]:
covar_scalers = []
for covar_col in model_spec["continuous_covariates"]:
    covar_scalers.append(StandardScaler())
    gdp_regression_data[covar_col.replace("[weight]",model_spec["weights"])+"_scaled"] = covar_scalers[-1].fit_transform(np.array(gdp_regression_data[covar_col.replace("[weight]",model_spec["weights"])]).reshape(-1,1)).flatten()
target_var_scaler = StandardScaler()
gdp_regression_data[model_spec["target"]+"_scaled"] = target_var_scaler.fit_transform(np.array(gdp_regression_data[model_spec["target"]]).reshape(-1,1)).flatten()

In [5]:
target_data = gdp_regression_data[model_spec["target"]+"_scaled"]
model_variables = []
for covar in model_spec["continuous_covariates"]:
    model_variables.append(covar.replace("[weight]",model_spec["weights"])+"_scaled")
for covar in model_spec["discrete_covariates"]:
    model_variables.append(covar)
for fe in model_spec["fixed_effects"]:
    for fe_col in [col for col in gdp_regression_data.columns if col.endswith(f"{fe}_fixed_effect")]:
        model_variables.append(fe_col)
for i in range(model_spec["incremental_effects"]):
    for ie_col in [col for col in gdp_regression_data.columns if col.endswith(f"incremental_effect_{i+1}")]:
        model_variables.append(ie_col)
model_data = gdp_regression_data[model_variables]

# Fixed-effects Bayesian model

In [34]:
with pm.Model() as pymc_model:

    model_variable_coefs = pm.Normal("model_variable_coefs", [-4,11,-7], [4,10,6], shape=(len(model_data.columns)))
    model_terms = pm.Deterministic("model_variable_terms", pt.sum(model_variable_coefs * model_data, axis=1))

    gdp_std_scale = pm.HalfNormal("gdp_std_scale", 5)
    gdp_std = pm.HalfNormal("gdp_std", sigma=gdp_std_scale)
    gdp_posterior = pm.Normal('gdp_posterior', mu=model_terms, sigma=gdp_std, observed=target_data)

    prior = pm.sample_prior_predictive()
    trace = pm.sample(target_accept=.99, cores=4)
    posterior = pm.sample_posterior_predictive(trace, extend_inferencedata=True)

Sampling: [gdp_posterior, gdp_std, gdp_std_scale, model_variable_coefs]
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [model_variable_coefs, gdp_std_scale, gdp_std]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 84 seconds.
Sampling: [gdp_posterior]


# Descaling example

In [35]:
for index, col in enumerate(model_data.columns):
    print(col, np.mean(np.mean(posterior.posterior.model_variable_coefs,axis=0), axis=0)[index].data)

fd_humidity_unweighted_scaled -4.258753109234552
fd_humidity_unweighted_2_scaled 10.52310459680783
fd_humidity_unweighted_3_scaled -6.917043049026962


In [36]:
model = sm.OLS(target_data,model_data)
covar_coef = model.fit().summary2().tables[1]

In [37]:
covar_coef

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
fd_humidity_unweighted_scaled,-4.347122,4.080492,-1.065343,0.292163,-12.556014,3.861769
fd_humidity_unweighted_2_scaled,10.737536,9.544939,1.124945,0.266324,-8.464404,29.939475
fd_humidity_unweighted_3_scaled,-7.053831,6.014311,-1.172841,0.246770,-19.153063,5.045401


In [38]:
model = sm.OLS(gdp_regression_data.fd_ln_gdp,gdp_regression_data[["fd_humidity_unweighted","fd_humidity_unweighted_3","fd_humidity_unweighted_2"]])
covar_coef = model.fit().summary2().tables[1]

In [39]:
covar_coef

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
fd_humidity_unweighted,-1.418614,1.794472,-0.790546,0.433180,-5.028627,2.191399
fd_humidity_unweighted_3,-0.009343,0.010476,-0.891845,0.377020,-0.030418,0.011732
fd_humidity_unweighted_2,0.219640,0.258373,0.850088,0.399586,-0.300140,0.739420


In [60]:
var1 = np.array(posterior.posterior.model_variable_coefs[:,:,0]).flatten()
var2 = np.array(posterior.posterior.model_variable_coefs[:,:,1]).flatten()
var3 = np.array(posterior.posterior.model_variable_coefs[:,:,2]).flatten()

var1_unscaled = (
    (var1 * np.std(gdp_regression_data.fd_ln_gdp) / np.std(gdp_regression_data.fd_humidity_unweighted)) 
    - ((2 * (var2 * np.mean(gdp_regression_data.fd_humidity_unweighted) * np.std(gdp_regression_data.fd_ln_gdp))) / np.std(gdp_regression_data.fd_humidity_unweighted_2))
    + ((3 * (var3 * np.mean(gdp_regression_data.fd_humidity_unweighted_2) * np.std(gdp_regression_data.fd_ln_gdp))) / np.std(gdp_regression_data.fd_humidity_unweighted_3))
)
print(np.mean(var1_unscaled))

var3_unscaled = var3 * np.std(gdp_regression_data.fd_ln_gdp) / np.std(gdp_regression_data.fd_humidity_unweighted_3)
print(np.mean(var3_unscaled))

var2_unscaled = (
    (var2 * np.std(gdp_regression_data.fd_ln_gdp) / np.std(gdp_regression_data.fd_humidity_unweighted_2))
    - (3 * (var3 * np.mean(gdp_regression_data.fd_humidity_unweighted) * np.std(gdp_regression_data.fd_ln_gdp) / np.std(gdp_regression_data.fd_humidity_unweighted_3)))
)
print(np.mean(var2_unscaled))

-1.7510407233482235
-0.011081937738102536
0.26286269705837284


In [42]:
np.mean(var1_unscaled)

-40.32178827023091

# Random-intercept Bayesian model

In [71]:
non_fe_vars = [col for col in model_data.columns if "year_fixed_effect" not in col and "country_fixed_effect" not in col]
country_fe_vars = [col for col in model_data.columns if "country_fixed_effect" in col]
year_fe_vars = [col for col in model_data.columns if "year_fixed_effect" in col]

with pm.Model() as pymc_model:

    non_fe_coefs = pm.Normal("model_variable_coefs", 0, 10, shape=(len(non_fe_vars)))
    non_fe_terms = pm.Deterministic("model_variable_terms", pt.sum(non_fe_coefs * model_data[non_fe_vars], axis=1))

    global_country_re_mean = pm.Normal("global_country_re_mean",0,10)
    global_country_re_sd = pm.HalfNormal("global_country_re_sd",5)
    country_re_means = pm.Normal("country_re_means", global_country_re_mean, global_country_re_sd, shape=len(country_fe_vars))
    country_re_sd = pm.HalfNormal("country_re_sd", 5)
    country_re_coefs = pm.Normal("country_re_coefs", country_re_means, country_re_sd, shape=len(country_fe_vars))
    country_re_terms = pm.Deterministic("country_re_terms", pt.sum(country_re_coefs * model_data[country_fe_vars], axis=1))

    global_year_re_mean = pm.Normal("global_year_re_mean",0,10)
    global_year_re_sd = pm.HalfNormal("global_year_re_sd",5)
    year_re_means = pm.Normal("year_re_means", global_year_re_mean, global_year_re_sd, shape=len(year_fe_vars))
    year_re_sd = pm.HalfNormal("year_re_sd", 5)
    year_re_coefs = pm.Normal("year_re_coefs", year_re_means, year_re_sd, shape=len(year_fe_vars))
    year_re_terms = pm.Deterministic("year_re_terms", pt.sum(year_re_coefs * model_data[year_fe_vars], axis=1))

    gdp_prior = pm.Deterministic(
        "gdp_prior",
        non_fe_terms +
        country_re_terms +
        year_re_terms
    )
    
    gdp_std_scale = pm.HalfNormal("gdp_std_scale", 5)
    gdp_std = pm.HalfNormal("gdp_std", sigma=gdp_std_scale)
    gdp_posterior = pm.Normal('gdp_posterior', mu=gdp_prior, sigma=gdp_std, observed=target_data)

    prior = pm.sample_prior_predictive()
    trace = pm.sample(target_accept=.99, cores=4)
    posterior = pm.sample_posterior_predictive(trace, extend_inferencedata=True)

Sampling: [country_re_coefs, country_re_means, country_re_sd, gdp_posterior, gdp_std, gdp_std_scale, global_country_re_mean, global_country_re_sd, global_year_re_mean, global_year_re_sd, model_variable_coefs, year_re_coefs, year_re_means, year_re_sd]
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...

KeyboardInterrupt



# Random-slopes Bayesian model

In [40]:
non_group_effect_vars = [col for col in model_data.columns if "year_fixed_effect" not in col and "country_fixed_effect" not in col and "incremental_effect" not in col]
incremental_effects = [col for col in model_data.columns if "incremental_effect" in col]
country_fe_vars = [col for col in model_data.columns if "country_fixed_effect" in col]
year_fe_vars = [col for col in model_data.columns if "year_fixed_effect" in col]

with pm.Model() as pymc_model:

    incremental_effect_coefs = pm.Normal("incremental_effect_coefs", 0, 10, shape=(len(incremental_effects)))
    incremental_effect_terms = pm.Deterministic("incremental_effect_terms", pt.sum(incremental_effect_coefs * model_data[incremental_effects], axis=1))

    global_country_rs_offset_mean = pm.Normal("global_country_rs_offset_mean",0,10)
    global_country_rs_offset_sd = pm.HalfNormal("global_country_rs_offset_sd",5)
    country_rs_offset_means = pm.Normal("country_rs_offset_means", global_country_rs_offset_mean, global_country_rs_offset_sd, shape=(len(non_group_effect_vars),len(country_fe_vars)))
    country_rs_offset_sd = pm.HalfNormal("country_rs_offset_sd", 5)
    country_rs_offsets = pm.Normal("country_rs_offsets", country_rs_offset_means, country_rs_offset_sd)

    country_rs_offset_columns = []
    for i in range(0,len(non_group_effect_vars)):
        country_rs_offset_columns.append(pt.sum(country_rs_offsets[i] * model_data[country_fe_vars],axis=1))
    country_rs_offset_matrix = pm.math.stack(country_rs_offset_columns, axis=1)
    
    global_year_rs_offset_mean = pm.Normal("global_year_rs_offset_mean",0,10)
    global_year_rs_offset_sd = pm.HalfNormal("global_year_rs_offset_sd",5)
    year_rs_offset_means = pm.Normal("year_rs_offset_means", global_year_rs_offset_mean, global_year_rs_offset_sd, shape=(len(non_group_effect_vars),len(year_fe_vars)))
    year_rs_offset_sd = pm.HalfNormal("year_rs_offset_sd", 5)
    year_rs_offsets = pm.Normal("year_rs_offsets", year_rs_offset_means, year_rs_offset_sd)

    year_rs_offset_columns = []
    for i in range(0,len(non_group_effect_vars)):
        year_rs_offset_columns.append(pt.sum(year_rs_offsets[i] * model_data[year_fe_vars],axis=1))
    year_rs_offset_matrix = pm.math.stack(year_rs_offset_columns, axis=1)

    covar_coefficients = pm.Normal("covar_coefficients", 0, 10, shape=len(non_group_effect_vars))
    covar_coef_matrix = pm.Deterministic("covar_coef_matrix", covar_coefficients * np.ones((len(gdp_regression_data), len(non_group_effect_vars))))  
    print(covar_coef_matrix.eval())
    covar_coefs_matrix_with_offset = pm.Deterministic("covar_coef_matrix_with_offset", pt.sum([covar_coef_matrix, country_rs_offset_matrix, year_rs_offset_matrix], axis=0))   
    print(country_rs_offset_matrix.eval())
    print(year_rs_offset_matrix.eval())
    print(covar_coefs_matrix_with_offset.eval())
    covar_terms = pm.Deterministic("country_fe_terms", pt.sum(covar_coefs_matrix_with_offset * model_data[non_group_effect_vars], axis=1))
    
    gdp_prior = pm.Deterministic(
        "gdp_prior",
        incremental_effect_terms +
        covar_terms
    )
    
    gdp_std_scale = pm.HalfNormal("gdp_std_scale", 5)
    gdp_std = pm.HalfNormal("gdp_std", sigma=gdp_std_scale)
    gdp_posterior = pm.Normal('gdp_posterior', mu=gdp_prior, sigma=gdp_std, observed=target_data)

    # prior = pm.sample_prior_predictive()
    # trace = pm.sample(target_accept=.99, cores=4)
    # posterior = pm.sample_posterior_predictive(trace, extend_inferencedata=True)

[[ 12.37870589   3.17858965   4.5534875  ...  -3.46436555 -14.61051546
    1.64741757]
 [ 12.37870589   3.17858965   4.5534875  ...  -3.46436555 -14.61051546
    1.64741757]
 [ 12.37870589   3.17858965   4.5534875  ...  -3.46436555 -14.61051546
    1.64741757]
 ...
 [ 12.37870589   3.17858965   4.5534875  ...  -3.46436555 -14.61051546
    1.64741757]
 [ 12.37870589   3.17858965   4.5534875  ...  -3.46436555 -14.61051546
    1.64741757]
 [ 12.37870589   3.17858965   4.5534875  ...  -3.46436555 -14.61051546
    1.64741757]]
[[-15.40158028  -8.38112755  -3.69093909 ... -12.73897116  -4.51151375
   -7.52881807]
 [-15.40158028  -8.38112755  -3.69093909 ... -12.73897116  -4.51151375
   -7.52881807]
 [-15.40158028  -8.38112755  -3.69093909 ... -12.73897116  -4.51151375
   -7.52881807]
 ...
 [-13.44147246  -6.21858906   2.10813171 ... -20.38600278 -14.79777341
  -10.98987118]
 [-13.44147246  -6.21858906   2.10813171 ... -20.38600278 -14.79777341
  -10.98987118]
 [-13.44147246  -6.21858906   2.